# imports

In [14]:
import sys
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, GRU, LSTM, Dropout
from keras.callbacks import  EarlyStopping, ModelCheckpoint

import emd

pd.options.plotting.backend = "plotly"
pd.set_option('precision', 2)

# funcs

In [45]:
def get_dat(site,filename=None,features='all',emd=True,rename=None,resample=None,start=None,end=None):
    if site == 'deere':
        df = pd.read_csv(  '/content/drive/MyDrive/Data/deere_load.csv',   
                              comment='#',
                              parse_dates=['Datetime (UTC-6)'],
                              index_col=['Datetime (UTC-6)'] )

    elif site == 'deere-supercleaned':
        df = pd.read_csv(  '/content/drive/MyDrive/Data/deere_load_supercleaned.csv',   
                              comment='#',
                              parse_dates=['Datetime (UTC-6)'],
                              index_col=['Datetime (UTC-6)'] )

    elif site == 'hyatt':
        df = pd.read_csv(  '/content/drive/MyDrive/Data/hyatt_load_IMFs.csv',   
                              comment='#',
                              parse_dates=['Datetime (UTC-10)'],
                              index_col=['Datetime (UTC-10)'] )       
      
    elif site == 'lajolla':
        df = pd.read_csv(  '/content/drive/MyDrive/Data/lajolla_load_IMFs.csv',   
                              comment='#',
                              parse_dates=['Datetime (UTC-8)'],
                              index_col=['Datetime (UTC-8)'] )  

    # elif site == 'nwe':
      #   df = pd.read_csv(   '/content/drive/MyDrive/Data/NWE/ca_actual.csv',   
      #                       comment='#',                 
      #                       parse_dates=['Date'],
      #                       index_col=['Date'])
      #   df = convert_nwe_data_to_vector(df)

    elif site == 'terna':
        file_path = '/content/drive/MyDrive/Data/terna_load_kw.csv'
        dfm = pd.read_csv(  file_path,
                            comment='#',
                            index_col=0)

        idx = pd.date_range(  start   = '2006-1-1 0:00',
                              end     = '2015-12-31 23:00',
                              freq    = 'H')

        df = pd.DataFrame(index=idx, data=np.empty(len(idx)), columns=0)

        begin, end = 0, 24
        for i in range(dfm.shape[0]):
            dat = dfm.iloc[i].values
            df.iloc[begin:end,0] = dat
            begin, end = begin+24, end+24

        # 2006-03-26 02:00:00   NaN
        # 2007-03-25 02:00:00   NaN
        # 2008-03-30 02:00:00   NaN
        # 2009-03-29 02:00:00   NaN
        # 2010-03-28 02:00:00   NaN
        # 2011-03-27 02:00:00   NaN
        # 2012-03-25 02:00:00   NaN
        # 2013-03-31 02:00:00   NaN
        # 2014-03-30 02:00:00   NaN
        # 2015-03-29 02:00:00   NaN
        df = df.fillna(method='ffill')
        df.iloc[:,0][df.iloc[:,0].isna()]
      
    else:
        df = pd.read_csv(filename,
                        comment='#',
                        index_col=0,
                        parse_dates=True)
        
    if start and end:
        df = df.loc[start:end,:]        
        
    if resample:
        df = df.resample(resample).mean()
        
    if rename:
        df = df.rename(columns={df.columns[0]:rename})
      
    if emd:
        df = emd_sift(df)
                                               
    # triangle waveform 
    df['Day'] = np.abs(df.index.dayofyear - 182)
    df['Hour'] = np.abs(np.abs(df.index.hour-12)-12)
    df['Weekday'] = np.abs(df.index.dayofweek - 3)
    
    dppd = {'H':24,'15T':96,'T':1440}[df.index.inferred_freq]
      
    d = df.iloc[:,0].values.flatten()
    rmse_np1d = rmse(d[(dppd*1):],d[:-(dppd*1)])
    rmse_np7d = rmse(d[(dppd*7):],d[:-(dppd*7)])

    if rmse_np1d < rmse_np7d:
        np_days = 1
    else:
        np_days = 7
    
    if features=='all':
        return df, dppd, np_days
    else:
        return df[features], dppd, np_days
  
def one_hot_of_peaks(ds,freq='D'):
    df = pd.DataFrame(ds)
    df['peak'] = 0
    df.loc[df.groupby(pd.Grouper(freq=freq)).idxmax().iloc[:,0], 'peak'] = 1  
    return df['peak']     

def accuracy_one_hot(true,pred):
    """ Measure the accuracy of two one hot vectors, inputs can be 1d numpy or dataseries"""
    n_misses = sum(true != pred)/2     # every miss gives two 'False' entries
    return 1 - n_misses/sum(true)   # basis is the number of one-hots


def batch_generator(batch_size, sequence_length, num_x_signals, num_y_signals,
                    num_train, X, Y):
    """
    Generator function for creating random batches of training-data.
    """    
    
    
    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = X[idx:idx+sequence_length]
            y_batch[i] = Y[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)     
        
def rmse(y_true, y_pred):
  return np.sqrt(np.mean(np.square(y_true - y_pred)))

def emd_sift(df):
  """ Sifts the left-mose column of dataframe"""
  imf = emd.sift.sift(df.iloc[:,0].values)

  for i in range(imf.shape[1]):
      df['IMF%s'%(i+1)] = imf[:,i]  

  return df           

# gru

In [51]:
# for units in [24,48,72,96,128]:
#     for sequence_length in [24,48,72,168]:
#         for layers in [1,2,3]:
#             for afuncs in [{'gru':'relu','dense':'relu'},{'gru':'softmax','dense':'relu'}]:
      
      
      
DEBUG = True      
# data
site = 'verizon'
filename = '~/data/verizon_west_excelsior_load_solarTMY.csv'
data_range = ['2021-9-1','2022-10-11']
dir = 'models'
features = ['Load (kW)',
            # #'NP',
            # 'IMF1',                                
            # 'IMF2',                                
            # 'IMF3',
            # 'IMF4',
            # 'IMF5',
            # 'IMF6',
            # 'IMF7',
            # 'IMF8',
            # 'IMF9',
            # 'IMF10',
            # 'IMF11',
            # 'Day',
            # 'Weekday',
            # 'Hour',    
            ]

# model
rnn_type='gru'
units=24
layers=1
dropout=0
afuncs={'lstm':'relu','gru':'relu','dense':'relu'}
loss='mse'
forecast = 'normal' # normal | peak
n_targets = 4


# training
targets =  ['LoadTarget'] # LoadTarget | LoadTargetOH
shift_steps = 1
sequence_length=4
epochs=10
patience=20
train_split = 0.9
batch_size=1
np.random.seed(42)
tf.random.set_seed(42)

# output
verbose=0
output = True
plots = False
#metrics = {'acc':[]}
metrics = {'mae':[],'mae_np':[],'skill':[]}

# model filename
t = datetime.now()
path_checkpoint = f'{dir}/{site}/{t.year}-{t.month:02}-{t.day:02}_' + \
                f'{t.hour:02}-{t.minute:02}-{t.second:02}_{rnn_type}-u{units}-l{layers}.keras'


# data
df,dppd,np_days = get_dat(site,
                          filename,
                          emd=True,
                          rename='Load (kW)',
                          resample='H',
                          start=data_range[0],
                          end=data_range[1])               

if forecast == 'peak':
    df['LoadOH'] =      one_hot_of_peaks(df[features[0]])
    df['LoadTargetOH'] =   one_hot_of_peaks(df[features[0]]).shift(-shift_steps)
    if 'NPOH' in features:
        df['NPOH'] =      one_hot_of_peaks(df[features[0]]).shift(np_days*dppd)
else:
    df['LoadTarget'] = df[features[0]].shift(-shift_steps)

if 'NP' in features:
    df['NP']         = df[features[0]].shift(np_days*dppd)

df = df.dropna()

# split and shape data
num_data = len(df)
num_train = int(train_split * num_data)
df_train = df.iloc[:num_train,:]
df_valid = df.iloc[num_train:,:]
X_scaler = MinMaxScaler()
X_train = X_scaler.fit_transform(df_train[features].values)
X_valid = X_scaler.fit_transform(df_valid[features].values)

# if DEBUG:
#     X_train = df_train[features].values
#     X_valid = df_valid[features].values

y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(df_train[targets].values)
y_valid = y_scaler.fit_transform(df_valid[targets].values)

# if DEBUG:
#     y_train = df_train[targets].values
#     y_valid = df_valid[targets].values

generator =    batch_generator( batch_size,
                                sequence_length,
                                num_x_signals=len(features),
                                num_y_signals=len(targets),
                                num_train=num_train,
                                X=X_train,
                                Y=y_train)   
X_valid = X_valid[np.newaxis,:,:]
y_valid = y_valid[np.newaxis,:,:] 

# if DEBUG:
#     X,y=next(generator)
#     print(X.shape,y.shape)

# build and train model
# convention: X and y are scaled values, 'Load' etc are not
if rnn_type == 'gru':
    model = Sequential()
    model.add( GRU( units=units,
                    return_sequences=True,
                    input_shape=(None, len(features),),
                    activation=afuncs[rnn_type]) )
    if layers > 1:
        model.add( GRU( units=units,
                        return_sequences=True,
                        activation=afuncs[rnn_type]) )
    if layers > 2:
        model.add( GRU( units=units,
                        return_sequences=True,
                        activation=afuncs[rnn_type]) )        
    model.add( Dense(units=n_targets,
                        activation=afuncs['dense']) )
elif rnn_type == 'lstm':
    if layers==1:
        model = Sequential([LSTM(units=units,
                                    return_sequences=True,
                                    input_shape=(None,len(features)),
                                    activation=afuncs[rnn_type]),
                            Dense(units=len(targets),
                                    activation=afuncs[rnn_type]) ] )
    elif layers==2:
        model = Sequential([LSTM(units=units,
                                    return_sequences=True,
                                    input_shape=(None,len(features)),
                                    activation=afuncs[rnn_type]),
                            LSTM(units=units,
                                    return_sequences=True,
                                    activation=afuncs[rnn_type]), 
                            Dense(units=n_targets,
                                    activation=afuncs[rnn_type]) ] )
model.compile( loss=loss, optimizer='adam')
model.summary()                
hx = model.fit( x=generator,
                epochs=epochs,
                steps_per_epoch=100,
                validation_data=(X_valid,y_valid),
                callbacks=[ ModelCheckpoint(filepath=path_checkpoint,
                                            monitor='val_loss',
                                            verbose=verbose,
                                            save_weights_only=True,
                                            save_best_only=True         ),
                            EarlyStopping  (monitor='val_loss',
                                            patience=patience,
                                            verbose=verbose             )   ])        

# eval
model.load_weights(path_checkpoint) # just in case, load best model
print('\n/// Best Model Loss (valid)\n',model.evaluate(X_valid,y_valid))

if not DEBUG:
    df_valid.loc[:,'Predictions'] =  y_scaler.inverse_transform(model.predict(X_valid).flatten()[:,np.newaxis]   )

    if forecast == 'peak':
        acc = accuracy_one_hot(df_valid.LoadTargetOH,one_hot_of_peaks(df_valid.Predictions))
        print('\n/// Accuracy\n',acc)
        metrics['acc'].append(acc)
    else:
        mae_np = df_valid.LoadTarget.diff(np_days*dppd).dropna().abs().mean()
        mae = (df_valid.LoadTarget - df_valid.Predictions).abs().mean()
        skill = 1 - mae/mae_np
        metrics['mae'].append(mae)
        metrics['mae_np'].append(mae_np)
        metrics['skill'].append(skill)
        print('\n/// MAE\n',mae)
        
    # save 
    pd.DataFrame(metrics).to_csv('results.csv')

    # plot
    pd.DataFrame(hx.history).plot()  

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_8 (GRU)                 (None, None, 24)          1944      
                                                                 
 dense_6 (Dense)             (None, None, 4)           100       
                                                                 
Total params: 2,044
Trainable params: 2,044
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
100/100 [==============================] - 1s 3ms/step - loss: 0.1596 - val_loss: 0.1726
Epoch 2/10
100/100 [==============================] - 0s 2ms/step - loss: 0.1125 - val_loss: 0.1529
Epoch 3/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0995 - val_loss: 0.1492
Epoch 4/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0984 - val_loss: 0.1483
Epoch 5/10
100/100 [============================

In [54]:
model.predict(X_valid).shape

1/1 [==============================] - 0s 40ms/step


(1, 975, 4)

In [52]:
df_valid

,Load (kW),Solar TMY (kW),IMF1,IMF2,IMF3,IMF4,IMF5,IMF6,IMF7,IMF8,IMF9,Day,Hour,Weekday,LoadTarget
Datetime PST,,,,,,,,,,,,,,,
2022-09-01 08:00:00,5.92,8.53,-0.08,-0.69,-0.06,0.49,-0.47,0.31,-0.04,0.21,6.27,62,8,0,6.68
2022-09-01 09:00:00,6.68,10.39,0.04,-0.05,-0.07,0.49,-0.47,0.31,-0.04,0.21,6.27,62,9,0,7.26
2022-09-01 10:00:00,7.26,11.55,-0.05,0.62,-0.07,0.49,-0.47,0.31,-0.04,0.21,6.27,62,10,0,8.00
2022-09-01 11:00:00,8.00,11.97,0.11,1.19,-0.08,0.50,-0.47,0.31,-0.05,0.21,6.27,62,11,0,8.49
2022-09-01 12:00:00,8.49,11.62,0.17,1.62,-0.08,0.50,-0.47,0.31,-0.05,0.21,6.27,62,12,0,8.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-11 18:00:00,6.55,0.00,-0.32,-0.44,1.46,-0.03,-0.10,0.33,-0.89,0.20,6.34,102,6,2,6.60
2022-10-11 19:00:00,6.60,0.00,0.21,-0.62,1.17,-0.02,-0.10,0.33,-0.89,0.20,6.34,102,5,2,6.16
2022-10-11 20:00:00,6.16,0.00,0.20,-0.69,0.81,-0.02,-0.10,0.33,-0.89,0.20,6.34,102,4,2,6.09


# sandbox

In [15]:
site = 'verizon'
filename = '~/data/verizon_west_excelsior_load_solarTMY.csv'
data_range = ['2021-9-1','2022-10-11']
df,dppd,np_days = get_dat(site,
                          filename,
                          emd=True,
                          rename='Load (kW)',
                          start=data_range[0],
                          end=data_range[1])                    
df['Load (kW)'].plot()